In [ ]:
import cv2
import pandas as pd
import numpy as np
from datetime import datetime
import os
from collections import deque
import requests
from ultralytics import YOLO

TOKEN = "7179015844:AAEdKPQgD6_DJk8krDzJHW_g5Ge6S795-fQ"
chat_id = "1383798128" #Jeevan
#chat_id = "1448781704"
caption = "A person is detected in your premises"
folder_path = "D:/vs code/pics"  # Specify the path to the folder containing images

class Tracker:
    def _init_(self):
        self.center_points = {}  # object id and center point values
        self.id_count = 0

    def update(self, objects_rect):
        objects_bbs_ids = []

        for rect in objects_rect:
            x, y, w, h, confidence = rect  # Extract confidence
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            same_object_detected = False
            for obj_id, pt in self.center_points.items():
                dist = np.linalg.norm(np.array([cx, cy]) - np.array(pt))  # euclidean distance
                if dist < 35:
                    self.center_points[obj_id] = (cx, cy)
                    objects_bbs_ids.append([x, y, w, h, confidence, obj_id])
                    same_object_detected = True
                    break

            if not same_object_detected:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, confidence, self.id_count])
                self.id_count += 1

        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, _, obj_id = obj_bb_id
            center = self.center_points[obj_id]
            new_center_points[obj_id] = center
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

def imgwrite(img):
    now = datetime.now()
    current_time = now.strftime("%d_%m_%Y_%H_%M_%S")
    filename = f"{current_time}.png"
    cv2.imwrite(os.path.join(folder_path, filename), img)

model = YOLO('yolov8s.pt')
tracker = Tracker()

#video input
cap = cv2.VideoCapture("D:/vs code/pathrakkaran.mp4")
#cap = cv2.VideoCapture("D:/vs code/night paalkaran.mp4")

with open("coco.txt", "r") as my_file:
    class_list = my_file.read().split("\n")

area = [(158, 478), (288, 290), (854, 265), (998, 470)]
#area = [(341, 347), (341, 119), (661, 119), (661, 347)] 
#area = [(311,290), (189,462), (595,468), (495,265)] 


area_c = set()  # detected objects id
prev_positions = {}
persons_inside_roi = set()  # id

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1020, 500))
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # to standardize color scheme

    px = pd.DataFrame(model.predict(frame_rgb)[0].boxes.data)  # object prediction
    obj_rect = []
    for _, row in px.iterrows():
        x1, y1, x2, y2, d, confidence = map(int, row[:6])  # Extract confidence score
        label = class_list[d]
        if label.startswith('person'):
            # Ensure the detected object is not too small to be a person
            if (x2 - x1) * (y2 - y1) > 1500:
                obj_rect.append([x1, y1, x2 - x1, y2 - y1, confidence])
        else:
            continue

    bbox_idx = tracker.update(obj_rect)
    for bbox in bbox_idx:
        x, y, w, h, conf, obj_id = bbox
        results = cv2.pointPolygonTest(np.array(area, np.int32), ((x + w//2, y + h)), False)  # diff between roi and bbox
        if class_list[d].startswith('person'):  # Check if the detected object is a person
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw bounding box only for persons
            cv2.circle(frame, (x + w//2, y + h), 4, (255, 0, 255), -1)  # Draw center point
            cv2.putText(frame, str(obj_id), (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)  # Write object ID

            if results >= 0:  # inside roi
                area_c.add(obj_id)
                persons_inside_roi.add(obj_id)

            if obj_id not in prev_positions:
                prev_positions[obj_id] = deque(maxlen=100)
            prev_positions[obj_id].appendleft((x + w//2, y + h))  # Modify to use foot position
            for i in range(1, len(prev_positions[obj_id])):
                if None in (prev_positions[obj_id][i - 1], prev_positions[obj_id][i]):
                    continue
                if cv2.pointPolygonTest(np.array(area, np.int32), (prev_positions[obj_id][i - 1][0], prev_positions[obj_id][i - 1][1]), False) >= 0 and \
                        cv2.pointPolygonTest(np.array(area, np.int32), (prev_positions[obj_id][i][0], prev_positions[obj_id][i][1]), False) >= 0:
                    cv2.line(frame, prev_positions[obj_id][i - 1], prev_positions[obj_id][i], (0, 255, 255), thickness=2)

            if obj_id in persons_inside_roi and results < 0:
                crop = frame[max(0, y - 150):min(frame.shape[0], y + h + 150),
                             max(0, x - 150):min(frame.shape[1], x + w + 150)]
                imgwrite(crop)

                # Send the captured image to Telegram
                photo_url = f'https://api.telegram.org/bot{TOKEN}/sendPhoto'
                for filename in os.listdir(folder_path):
                    if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                        photo_path = os.path.join(folder_path, filename)
                        data = {'chat_id': chat_id, 'caption': caption}
                        with open(photo_path, 'rb') as photo:
                            files = {'photo': photo}
                            response = requests.post(photo_url, data=data, files=files)
                            print(response.json())

                persons_inside_roi.remove(obj_id)
        else:
            continue
    cv2.polylines(frame, [np.array(area, np.int32)], True, (255, 0, 0), 2)

    k = len(area_c)
    cv2.putText(frame, str(k), (50, 60), cv2.FONT_HERSHEY_PLAIN, 5, (255, 0, 0), 3)

    cv2.imshow("RGB", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
yolo = YOLO('yolov8m.pt')

def detect_humans_in_video(video_path):
    cap = cv2.VideoCapture(video_path)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = yolo(frame)

        for result in results[0].boxes:
            if result.cls == 0:  # 0 is the class index for humans in YOLOv8
                x1, y1, x2, y2 = result.xyxy[0]
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

        cv2.imshow('Human Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Provide the path to the video file
video_path = 'WhatsApp Video 2024-05-07 at 1.29.43 PM.mp4'
detect_humans_in_video(video_path)

In [ ]:
model.classes